# Quick Start

## Step 1: Install Requirements

In [ ]:
!pip install -r requirements.txt

### Step 2: Define Your AutoInnovator

An **AutoInnovator** operates in a loop, where each iteration—called a **generation**—produces a set of **candidate algorithms**. The process for each candidate is:

1. **Create Prompt** – Your defined `create_prompt_kwargs` is used to construct a request to send to a Large Language Model (LLM)
   * See https://platform.openai.com/docs/api-reference/responses/create for what can be included in your request
   * See `Context` and `Candidate` classes in [autoinnovator/framework.py](autoinnovator/framework.py) for what historic data you can access
3. **Send Prompt to LLM** – The prompt is submitted to a LLM.
4. **Extract Algorithm Code** – Your defined `extract_algorithm_code` is used to extract algorithm code from the LLM’s response.
5. **Evaluate Algorithm** – The candidate algorithm is tested and scored.

Each candidate is generated independently but has access to the full history of prior generations. The goal is to design your AutoInnovator in a way that uses data from previous generations to guide the creation of higher-performing algorithms over generations.

> ⚠️ **Important:** The AutoInnovator should not directly edit or mutate algorithms. Instead, it evolves its approach by refining prompts and strategies for generating new candidates.


In [ ]:
from autoinnovator import AutoInnovatorBase, Candidate, Context
import os

class SimpleAutoInnovator(AutoInnovatorBase):
    def system_prompt(self, ctx: Context) -> str:
        with open(os.path.join("example_prompts", f"{ctx.challenge.name}.txt"), "r") as f:
            prompt = f.read()
        return prompt.format(
            parameters=ctx.challenge.parameters,
            base_algorithm=ctx.challenge.base_algorithm
        )

    def create_prompt_kwargs(self, candidate: Candidate, ctx: Context) -> dict:
        if candidate.generation == 1:
            prev_candidate = ctx.candidates[0][0] # 0th gen just has 1 candidate using base algorithm
        else:
            prev_candidate = ctx.candidates[candidate.generation - 1][candidate.id] # just use 1 candidate from previous generation
        if not prev_candidate.success:
            # Another strategy would be to use the previous generation's best candidate
            raise RuntimeError("Previous candidate was not successful. Simple Innovator strategy cannot continue.")
        algorithm = prev_candidate.algorithm
        response = prev_candidate.response
        if response:
            prev_response_id = response["id"]
            text = response["output"][0]["content"][0]["text"]
            reasoning = text.split('<reasoning>')[1].split('</reasoning>')[0]
        else:
            prev_response_id = None
            reasoning = "No reasoning yet. Evaluation was on base algorithm."
        evaluation = prev_candidate.evaluation
        evaluation["algorithm_code_length"] = len(algorithm)
        
        return {
            "instructions": self.system_prompt(ctx),
            "input": f"ALGORITHM:\n{algorithm}\nREASONING:\n{reasoning}\nEVALUATION:\n{evaluation}",
            "temperature": 1.0,
            "previous_response_id": prev_response_id,
        }

    def extract_algorithm_code(self, response: dict) -> str:
        text = response["output"][0]["content"][0]["text"]
        if "<python>" in text and "</python>" in text:
            return text.split("<python>")[1].split("</python>")[0].strip()
        raise ValueError("Response does not contain valid algorithm code.")

## Step 3: Run Your AutoInnovator on the Challenges

We’ve provided **three built-in challenges** to test your AutoInnovator:

1. **KDE** – Generate parameters for a Gaussian Mixture Model that best fits samples drawn from an intractable target distribution.
2. **Binning** – Generate histogram bin edges within the interval $[-1, 1]$ that best represent samples from an intractable target distribution over $[-1, 1]$.
3. **Clustering** – Assign samples from an intractable target distribution into $K$ clusters (where $K$ is provided) to maximize the **pairwise cluster connectivity score**.

> ⚠️ You should not modify the challenges themselves when developing your AutoInnovator. However, you’re welcome to inspect the challenge code to better understand the setup.

### Challenge Code Structure

Each challenge is organized into the following components:
1. `challenges/<challenge_name>.py` → Defines the challenge as an executable Python script.
2. `challenges/configs/<challenge_name>.json` → Specifies configuration settings for generating challenge instances.  Each candidate is evaluated against the same fixed set of instances.
3. `challenges/base_algos/<challenge_name>_<algo_name>.py` → Provides a baseline algorithm implementation for solving challenge instances.


In [ ]:
API_KEY = None
if not API_KEY:
    raise ValueError("You must set an OpenAI API Key")

from autoinnovator import Challenge, LLM, LLMProvider
llm = LLM(
    provider=LLMProvider.OPENAI,
    model="gpt-4.1-mini-2025-04-14",
    api_key=API_KEY,
)
my_autoinnovator = SimpleAutoInnovator()
kde_challenge = Challenge("kde")
binning_challenge = Challenge("binning")
clustering_challenge = Challenge("clustering")

### 3.1 Running KDE Challenge

In [ ]:
from IPython.display import Image, display, clear_output, Markdown
import time

num_visualisations = 4
def on_generation_done(ctx: Context):
    clear_output(wait=True)
    best_candidate = min(
        filter(lambda c: c.success, ctx.candidates[ctx.curr_generation]),
        key=lambda c: c.evaluation["test_log_likelihood_average"]
    )
    display(Markdown(f"### Generation {ctx.curr_generation} - Best Candidate {best_candidate.id}"))
    for i in range(num_visualisations):
        display(Image(filename=best_candidate.visualisation_path.format(i=i)))
    display(Image(filename=ctx.results_plot_path))
    time.sleep(1)

kde_ctx = my_autoinnovator.run(
    llm=llm,
    challenge=kde_challenge,
    num_generations=10,
    candidates_per_generation=1,
    experiment_dir=os.path.join("experiments", "kde"),
    num_visualisations_per_candidate=num_visualisations,
    on_generation_done=on_generation_done
)

### 3.2 Running Binning Challenge

In [ ]:
from IPython.display import Image, display, clear_output, Markdown
import time

num_visualisations = 4
def on_generation_done(ctx: Context):
    clear_output(wait=True)
    best_candidate = min(
        filter(lambda c: c.success, ctx.candidates[ctx.curr_generation]),
        key=lambda c: c.evaluation["test_log_likelihood_average"]
    )
    display(Markdown(f"### Generation {ctx.curr_generation} - Best Candidate {best_candidate.id}"))
    for i in range(num_visualisations):
        display(Image(filename=best_candidate.visualisation_path.format(i=i)))
    display(Image(filename=ctx.results_plot_path))
    time.sleep(1)

binning_ctx = my_autoinnovator.run(
    llm=llm,
    challenge=binning_challenge,
    num_generations=10,
    candidates_per_generation=1,
    experiment_dir=os.path.join("experiments", "binning"),
    num_visualisations_per_candidate=num_visualisations,
    on_generation_done=on_generation_done
)

### 3.3 Running Clustering Challenge

In [ ]:
from IPython.display import Image, display, clear_output, Markdown
import time

num_visualisations = 4
def on_generation_done(ctx: Context):
    clear_output(wait=True)
    best_candidate = min(
        filter(lambda c: c.success, ctx.candidates[ctx.curr_generation]),
        key=lambda c: c.evaluation["cluster_scores_average"]
    )
    display(Markdown(f"### Generation {ctx.curr_generation} - Best Candidate {best_candidate.id}"))
    for i in range(num_visualisations):
        display(Image(filename=best_candidate.visualisation_path.format(i=i)))
    display(Image(filename=ctx.results_plot_path))
    time.sleep(1)

clustering_ctx = my_autoinnovator.run(
    llm=llm,
    challenge=clustering_challenge,
    num_generations=10,
    candidates_per_generation=1,
    experiment_dir=os.path.join("experiments", "clustering"),
    num_visualisations_per_candidate=num_visualisations,
    on_generation_done=on_generation_done
)